<a href="https://colab.research.google.com/github/TimmyMorio/stockpublic/blob/main/Stock_Alert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Email configuration
EMAIL_USER = 'timshurmelev@mail.ru'
EMAIL_PASSWORD = 'tgjcSvQNwTUpT5XAfZnk'

# Function to send email notifications
def send_email(subject, body, to_email):
    from_email = EMAIL_USER
    password = EMAIL_PASSWORD

    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    try:
        with smtplib.SMTP('smtp.mail.ru', 587) as server:
            server.starttls()
            server.login(from_email, password)
            text = msg.as_string()
            server.sendmail(from_email, to_email, text)
        print(f"Email sent to: {to_email}")
    except Exception as e:
        print(f"Error sending email: {e}")

# Function to calculate support lines
def calculate_support(data, window=90):
    data['Support'] = data['Low'].rolling(window=window).min()
    return data

# Function to identify stocks near or below support line
def check_support_break(stock_list, window=20, threshold=0.01):
    result = []
    failed_tickers = []
    for stock in stock_list:
        try:
            # Download historical data
            data = yf.download(stock, period='1y', interval='1d')

            # Check if data is empty
            if data.empty:
                print(f"No data for {stock}. Skipping...")
                failed_tickers.append(stock)
                continue

            # Calculate support levels
            data = calculate_support(data, window)

            # Ensure Support column has valid data
            if data['Support'].isna().all():
                print(f"Support data not available for {stock}. Skipping...")
                failed_tickers.append(stock)
                continue

            # Get the last close and support values
            last_close = data['Close'].iloc[-1]
            last_support = data['Support'].iloc[-1]

            # Check if last_close is less than or equal to the support level within threshold
            if last_close <= last_support * (1 + threshold):
                result.append({
                    'Stock': stock,
                    'Close': last_close,
                    'Support': last_support,
                    'Break': last_close < last_support
                })

                # Send email notification
                subject = f"Alert: {stock} Close to Support Line"
                body = f"{stock} is close to its support level. Current close: {last_close}, Support: {last_support}"
                send_email(subject, body, "timshurmelev@mail.ru")

        except Exception as e:
            print(f"Unexpected error for {stock}: {e}")
            failed_tickers.append(stock)

    return pd.DataFrame(result), failed_tickers

# Define your stock list
stock_list = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'NVDA', 'GOOGL', 'META', 'BRK-B', 'JNJ', 'PG',
    'V', 'MA', 'UNH', 'WMT', 'JPM', 'HD', 'INTC', 'KO', 'PEP', 'NFLX',
    'ABBV', 'ADBE', 'CSCO', 'T', 'VZ', 'PFE', 'CRM', 'MRK', 'ORCL', 'IBM',
    'XOM', 'CVX', 'DIS', 'MCD', 'PYPL', 'SBUX', 'NKE', 'GE', 'BA', 'GILD',
    'TXN', 'QCOM', 'LMT', 'MMM', 'ABT', 'HON', 'DHR', 'MDT', 'GS', 'MS',
    'AMT', 'TMUS', 'BMY', 'CVS', 'GM', 'F', 'AXP', 'BIDU', 'BABA', 'JD',
    'NIO', 'LI', 'XPEV', 'ZM', 'SQ', 'TWLO', 'PTON', 'SNOW', 'SHOP',
    'RBLX', 'PLTR', 'DKNG', 'ETSY', 'DASH', 'U', 'MDB', 'OKTA', 'NOW',
    'HUBS', 'VEEV', 'DDOG', 'CPNG', 'AI', 'FVRR', 'CRWD', 'WIX', 'Z',
    'UBER', 'LYFT', 'BYND', 'NTNX', 'FSLY', 'SQSP', 'ALNY',
    'ILMN', 'SPGI', 'NDAQ']

# Run the check
results, failed_tickers = check_support_break(stock_list)

# Show results
print(results)

# Show failed tickers
if failed_tickers:
    print("\nFailed tickers:")
    for ticker in failed_tickers:
        print(ticker)